In [ ]:
1.what is terraform and how its work?

In [ ]:
Terraform is an Infrastructure as Code (IaC) tool that allows us to provision, configure, and manage infrastructure across multiple cloud providers like AWS, Azure, GCP, and on-prem.

It works in a declarative way:

We define the desired infrastructure in .tf files (e.g., VMs, networks, storage).

Terraform builds a plan by comparing the current state (from the state file + cloud) with the desired state.

On terraform apply, it executes changes to bring real infrastructure in sync with the code.

It maintains a state file to track resources and dependencies.

In [ ]:
2. A Devops Engineer manually created infrastructure on aws, and now there is a requirement to use terraform to manage it. how would you import these resources in terraform code?

In [ ]:
--> If resources are already created manually in AWS, Terraform can still manage them using the terraform import command.
Steps I would follow:

Write the Terraform resource block in .tf file (matching the AWS resource type). For example:

resource "aws_instance" "my_ec2" {
  # arguments will be filled later
}

Run terraform import with the resource name and the AWS resource ID. Example:

terraform import aws_instance.my_ec2 i-0123456789abcdef0

After import, Terraform links that AWS resource to the state file.

Then I run terraform plan – it may show differences because the .tf file is empty.

Use terraform show or terraform state show to capture the actual configuration and update my .tf files accordingly.

Once .tf matches the real infra, Terraform can manage it going forward.

In [ ]:
3.you have multiple environments -dev,stage, prod for your application and you want to use the same code for all of these environment. how can you do that?

In [ ]:
"I keep the Terraform code common, and manage differences with variables and separate tfvars files 
(or workspaces). This way I can reuse the same code for dev, stage, and prod while isolating their state files."


In [ ]:
4.what is terraform state file? why it is importent?

In [ ]:
--> *"The Terraform state file (terraform.tfstate) is the single source of truth for all the infrastructure managed by Terraform. It maps the resources defined in code to the actual cloud resources, storing their IDs, attributes, and dependencies.

It’s crucial because Terraform uses it to:

Track Infrastructure: It knows exactly what exists and prevents accidental duplicates.

Plan Accurate Changes: Terraform compares the desired configuration with the state to generate precise execution plans.

Maintain Dependencies: Ensures resources are created, updated, or destroyed in the right order.

Enable Collaboration: Using remote state, multiple engineers can safely work on the same infrastructure without conflicts.

Detect Drift & Ensure Idempotency: Any out-of-band changes in the infrastructure are detected, and repeated runs of Terraform don’t break the environment.

In short, without the state file, Terraform loses control over the infrastructure. Managing it proactively—using remote backends, state locking, and versioning—is essential for a safe, scalable, and team-friendly DevOps workflow."*

In [ ]:
5.Jr Devops engineer accidently deleted the state file, what steps should we take to resolve it?

In [ ]:
"If a state file is deleted, first restore it from backup.
 If no backup exists, we rebuild the state using terraform import for all existing resources.
 Going forward, we should use remote state with versioning and locking to prevent accidental loss."

In [ ]:
6.what are some best practices to manage terraform state file?

In [ ]:
Best practices for Terraform state are: use remote backends, 
enable state locking and versioning, encrypt and secure the state file, 
separate environments with different state files, and never edit state manually. 
This ensures safe, scalable, and collaborative infrastructure management."

In [ ]:
7. there are some bash scripts thats you want to run after creating your resources with terraform so how would you achieve this

In [ ]:
How to run Bash scripts after creating resources in Terraform

Using provisioners (local-exec / remote-exec)

Terraform has provisioners that can run commands or scripts after a resource is created.

Example: Run a bash script on your local machine after creating an EC2 instance:

resource "aws_instance" "web" {
  ami           = "ami-123456"
  instance_type = "t2.micro"

  provisioner "remote-exec" {
    inline = [
      "sudo apt-get update -y",
      "sudo apt-get install -y nginx"
    ]

    connection {
      type     = "ssh"
      user     = "ec2-user"
      private_key = file("~/.ssh/id_rsa")
      host     = self.public_ip
    }
  }
}


Or run a local bash script:

provisioner "local-exec" {
  command = "bash ./scripts/post_deploy.sh"
}


Using null_resource + provisioners

If the script is not tied to a specific resource, you can use a null_resource.

Example:

resource "null_resource" "post_setup" {
  provisioner "local-exec" {
    command = "bash ./scripts/setup.sh"
  }

  depends_on = [aws_instance.web]
}


Here, Terraform ensures the EC2 instance is created before running the script.

Better Practice – Use Configuration Management

Provisioners should be a last resort (Terraform docs say this).

In real-world setups, we often use:

User data (for AWS EC2 to run bash scripts on boot).

Configuration management tools like Ansible, Chef, or Puppet for post-deployment scripts.

🎯 Interview-style Answer

"To run bash scripts after creating resources, I can use Terraform provisioners like local-exec or remote-exec. If the script isn’t tied to a resource, I’d use a null_resource with depends_on. However, in real-world practice, I prefer user data or a config management tool like Ansible, since provisioners should be a last resort."

In [ ]:
8. when you created a environment using terraform  what are the all components  which you have created using terraform

In [ ]:
When I create an environment using Terraform, 
I start with the provider block to connect to the cloud, 
then configure a remote backend for state management. 
Next, I define all infrastructure components like networking (VPC, subnets, gateways), compute (EC2, load balancers, autoscaling), storage (S3, RDS, EFS), 
IAM for security, 
and monitoring/logging with CloudWatch. 
I also use variables and outputs to make the code reusable across dev, stage, and prod. This way, the whole environment is automated, modular, and consistent.

In [ ]:
9.how to change the configaration alredy created resources using terraform

In [ ]:
"If the resource is already managed by Terraform, 
I just update the code and run plan + apply. 
But if the resource was created manually, I use terraform import to bring it under Terraform’s control. 
After importing, I can safely change its configuration through Terraform like any other resource."

In [ ]:
10.when you run terraform code it will generate a state file how you can manage it and store it?

In [ ]:
How Terraform State File is Managed & Stored
1. Local State (default)

By default, Terraform creates terraform.tfstate locally in the working directory.

This works for testing or single-user setups, but is not safe for teams.

Risk: file can be deleted, corrupted, or overwritten.

2. Remote State (Best Practice for Teams)

In production, we store the state file in a remote backend so multiple team members can work safely.

Common backends:

AWS S3 + DynamoDB → (S3 for state storage, DynamoDB for state locking)

Terraform Cloud / Enterprise → built-in remote backend with locking & versioning

Azure Blob Storage

Google Cloud Storage (GCS)

3. Example: Remote State with AWS S3 + DynamoDB
terraform {
  backend "s3" {
    bucket         = "my-terraform-states"
    key            = "prod/app/terraform.tfstate"
    region         = "us-east-1"
    dynamodb_table = "terraform-locks"   # Prevents concurrent changes
    encrypt        = true                # Encrypt state at rest
  }
}


S3 bucket stores the state file.

DynamoDB ensures only one apply runs at a time (state locking).

Versioning in S3 lets us roll back if state is corrupted/deleted.

4. Best Practices for Managing State

Always use remote backend for collaboration.

Enable state locking (DynamoDB, Terraform Cloud).

Enable versioning in the backend storage (S3, GCS).

Restrict access using IAM policies.

Never edit state manually — use terraform state mv or terraform state rm instead.

In [ ]:
11. if you lose the sate file how you resolve the issue?

In [ ]:
If the state file is lost, the first step is to restore it from a remote backend or backup. 
If no backup is available, 
I would rebuild the state using terraform import for each existing resource.
To avoid this issue in the future, I’d store state in a remote backend like S3 with versioning and DynamoDB locking enabled."

In [ ]:
12. features of terraform?

In [ ]:
"Terraform is an Infrastructure-as-Code tool that lets you define your infrastructure declaratively. Its key features are:

Execution plans to preview changes before applying.

State management to track resources and detect drift.

Automatic dependency handling with a resource graph.

Multi-cloud support with a wide range of providers.

Reusable modules for consistent infrastructure across environments.

Together, these features make infrastructure provisioning predictable, scalable, and easy to manage."

In [ ]:
13.what is terraform validate? why its used?

In [ ]:
"Terraform validate is a CLI command used to check the syntax and correctness of Terraform code without applying it.
 It helps catch errors early, ensures variables and resources are properly defined, 
 and is useful in CI/CD pipelines to prevent invalid configurations from being applied."
 Terraform validate checks for syntax errors like missing curly braces,
  undefined variables, 
  missing required attributes in resources, 
  wrong module paths, and incorrect expressions. 
  It ensures the code is structurally correct before planning or applying changes."

In [ ]:
14.life cycle in terraform?

In [ ]:
Lifecycle Meta-Arguments

create_before_destroy

Ensures a new resource is created before the old one is destroyed.

Useful for resources where downtime is not acceptable (e.g., load balancers, databases).

resource "aws_instance" "web" {
  ami           = "ami-123456"
  instance_type = "t2.micro"

  lifecycle {
    create_before_destroy = true
  }
}


prevent_destroy

Prevents Terraform from destroying a resource, even if it’s removed from the code.

Useful for critical resources like production databases or S3 buckets.

lifecycle {
  prevent_destroy = true
}


ignore_changes

Tells Terraform to ignore changes to specific attributes made outside Terraform.

Useful when some attributes are managed manually or by another tool.

lifecycle {
  ignore_changes = [tags["Environment"]]
}

2. Why It’s Important

Helps avoid downtime during updates.

Protects critical resources from accidental deletion.

Supports hybrid management where some attributes are changed outside Terraform.

"Terraform lifecycle controls resource behavior during create, update, and destroy.
create_before_destroy ensures zero-downtime updates,
prevent_destroy protects critical resources from accidental deletion,
ignore_changes lets Terraform ignore manual changes to specific attributes."

In [ ]:
15.what is the differences between terraform and ansible? when we use terraform when we use ansible?

In [ ]:
"Terraform and Ansible are both DevOps tools but serve different purposes.
 Terraform is used to provision and manage cloud infrastructure declaratively — like VPCs, EC2, S3, RDS
— and it keeps track of resources using a state file. Ansible, 
on the other hand, is used to configure and manage software on servers 
— like installing packages, setting up applications, or updating configurations — in an idempotent, 
 task-based way.

In [ ]:
16.we need to destroy one resource  in terraform is it possible ?

In [ ]:
Yes — Terraform allows destroying a single resource without affecting the rest of your infrastructure. Here’s how you can explain it clearly in an interview:

1. Using terraform destroy -target

You can target a specific resource to destroy:

terraform destroy -target=aws_instance.web


Terraform will only destroy the aws_instance.web resource and leave all other resources intact.

2. Using terraform state rm (Alternative)

If you want Terraform to stop managing a resource without destroying it in the cloud:

terraform state rm aws_instance.web


This removes the resource from the state file; the actual cloud resource remains.

Interview-Ready Answer:

"Yes, it’s possible to destroy a single resource in Terraform using terraform destroy -target=<resource> which only destroys the specified resource. Alternatively, terraform state rm <resource> can be used to remove a resource from state without deleting it."

In [ ]:
17. how you manage secreate in terraform?

In [ ]:
Use Sensitive Variables

Mark variables as sensitive = true so they won’t be displayed in CLI output or logs.

variable "db_password" {
  type      = string
  sensitive = true
}


Example usage:

resource "aws_db_instance" "mydb" {
  username = "admin"
  password = var.db_password
}

2. Store Secrets Securely

Environment Variables:

export TF_VAR_db_password="MySecret123"


Secret Management Tools: Use AWS Secrets Manager, HashiCorp Vault, or Parameter Store.

data "aws_secretsmanager_secret_version" "db_pass" {
  secret_id = "prod/db-password"
}

resource "aws_db_instance" "mydb" {
  username = "admin"
  password = data.aws_secretsmanager_secret_version.db_pass.secret_string
}

3. Protect State Files

Terraform state files can contain sensitive values. To secure them:

Use remote backend (S3, Terraform Cloud, GCS)

Enable encryption (encrypt = true)

Enable state locking to prevent concurrent changes

terraform {
  backend "s3" {
    bucket         = "my-terraform-states"
    key            = "prod/app/terraform.tfstate"
    region         = "us-east-1"
    encrypt        = true
    dynamodb_table = "terraform-locks"
  }
}

Interview-Ready Answer:

"To preserve keys in Terraform, I mark variables as sensitive = true, use environment variables or secret managers like AWS Secrets Manager for storing secrets, and store the Terraform state in a secure, encrypted remote backend. This ensures secrets are never exposed in CLI output or logs and the state file remains secure."

In [ ]:
18.  we deleted the state file what will happened we again do terraform plan , apply?

In [ ]:
What Happens If State File Is Deleted

Terraform loses track of all managed resources.

The state file is what Terraform uses to know which resources exist and their attributes.

If you run terraform plan or terraform apply after deleting the state:

Terraform will treat all existing resources as new.

It will try to create all resources again, which may cause conflicts or duplicate resources in the cloud.

In [ ]:
19. different kinds of modules in terraform?

In [ ]:
Terraform Modules

A module is a container for multiple resources that are used together.

Root module → The main .tf files in your working directory.

Child module → Any module called by another module.

Types of Modules in Terraform

Root Module

The main working directory where you run terraform init/apply.

Example: All your .tf files in prod/ or dev/ folder.

Child / Local Module

A module stored locally and reused within your project.

Example: modules/vpc/main.tf called from env/prod/main.tf

module "vpc" {
  source = "../modules/vpc"
  cidr_block = "10.0.0.0/16"
}


Registry Module

Modules downloaded from the Terraform Registry (public or private).

Example:

module "consul" {
  source  = "hashicorp/consul/aws"
  version = "0.10.0"
}


External / Published Module

Hosted externally, e.g., on GitHub, Bitbucket, or private repo.

Example:

module "vpc" {
  source = "git::https://github.com/org/vpc-module.git"
}

Why Use Modules

Reusability: Write once, use in multiple environments.

Maintainability: Organize large infrastructures.

Consistency: Enforce standard practices across teams.

Interview-Ready Answer

"Terraform modules are reusable containers for resources. The main types are: root modules (the main working directory), local/child modules (reused locally), registry modules (downloaded from Terraform Registry), and external modules (hosted in GitHub or private repos). Modules help make infrastructure code reusable, consistent, and maintainable."

In [ ]:
20.remote backend in terraform?

In [ ]:
A backend in Terraform determines where the state file is stored.

By default, Terraform uses a local backend (state file stored locally).

A remote backend stores the state in a centralized location, which is important for team collaboration and security.

Benefits of Remote Backend

Collaboration: Multiple team members can work on the same infrastructure safely.

State Locking: Prevents concurrent runs that could corrupt the state.

Versioning: Many remote backends support versioning, allowing rollback of state.

Security: State files can contain sensitive data; remote backends often support encryption.

Common Remote Backends

AWS S3 + DynamoDB (state in S3, locking via DynamoDB)

Terraform Cloud / Enterprise (built-in locking & versioning)

Azure Blob Storage

Google Cloud Storage (GCS)

Example: AWS S3 Remote Backend
terraform {
  backend "s3" {
    bucket         = "my-terraform-states"
    key            = "prod/app/terraform.tfstate"
    region         = "us-east-1"
    encrypt        = true
    dynamodb_table = "terraform-locks"  # For locking
  }
}


Explanation:

bucket → S3 bucket to store the state

key → Path inside the bucket

encrypt = true → Encrypt state at rest

dynamodb_table → Prevent concurrent apply runs

Interview-Ready Answer

"A remote backend in Terraform stores the state file in a centralized location like S3, GCS, or Terraform Cloud. It enables collaboration, state locking, versioning, and encryption, making infrastructure management safer and more reliable for teams."

In [ ]:
21.how do you pass variable in run time in terraform?

In [ ]:
Ways to Pass Variables at Runtime in Terraform
1. Using -var Flag

Pass variables directly in the CLI during plan or apply.

terraform plan -var="instance_type=t2.micro" -var="env=prod"
terraform apply -var="instance_type=t2.micro"

2. Using .tfvars File

Create a file, e.g., prod.tfvars:

instance_type = "t2.micro"
env = "prod"


Pass it during Terraform run:

terraform plan -var-file="prod.tfvars"
terraform apply -var-file="prod.tfvars"

3. Using Environment Variables

Prefix variable with TF_VAR_ in your shell:

export TF_VAR_instance_type="t2.micro"
export TF_VAR_env="prod"
terraform apply

4. Default Variables

You can also define defaults in variables.tf, but they can be overridden at runtime using any of the above methods.

variable "instance_type" {
  type    = string
  default = "t2.small"
}

Interview-Ready Answer

"In Terraform, variables can be passed at runtime using the -var flag, through a .tfvars file, or via environment variables prefixed with TF_VAR_. This allows dynamic configura

In [ ]:
22.in your organization which terraform version your using currently?

In [ ]:
"In my current/previous organization, we are using Terraform v1.x (for example, v1.5.7).
 We chose this version because it provides stable features, improved provider dependency management, 
 better CLI commands, and full backward compatibility with our existing infrastructure modules.
  For any critical environments, we always lock the Terraform version in required_version in terraform block to ensure consistency across teams."

In [ ]:
22. Drawbacks of Terraform

In [ ]:


State File Management – Losing or corrupting the state file can break infrastructure; requires careful handling with remote backends and versioning.

Steep Learning Curve – HCL syntax, modules, workspaces, and state concepts can be tricky for beginners.

Limited Procedural Logic – Being declarative, complex conditional logic or loops are harder compared to imperative tools.

Provider Limitations – Some cloud providers or services may not have full Terraform support or lag behind new features.

Secrets Management – Sensitive data can be exposed if state files are not encrypted; often requires integration with Vault or cloud secret managers.

Dependency on Other Tools – To configure applications inside VMs, Terraform usually relies on tools like Ansible, Chef, or shell scripts.

In [ ]:
23. terraform init, plan , apply, destroy?

In [ ]:
"Terraform init initializes the working directory and downloads providers.
 Terraform plan shows a preview of changes without applying them.
  Terraform apply executes the changes to reach the desired state,
   and terraform destroy removes all resources managed by Terraform."